# Visitor

## O que é?

O padrao Visitor permite que uma nova operacao seja efetuada sobre uma classe ou um grupo de classes sem que essas classes precisem mudar. Dessa maneira, ele promove a separacao entre o método em si e o objeto sobre o qual esse método opera.

## Por quê?

Para manter o "single responsibility principle" e evitar fazer grandes mudancas em classes que já estao em uso, há situacoes em que pode ser interessante permitir que uma nova funcionalidade seja acrescentada a uma classe sem que a classe em si seja responsável pela lógica -- particularmente se a funcionalidade será adicionada a mais de uma classe. 

## Estrutura

<img src="http://coursegalaxy.com/designpatterns/visitor/images/visitor_structure.jpg" alt="Visitor Design Pattern UML Class Diagram.svg">

## Exemplo

Imaginemos que tenhamos três classes: `Input`, `Image` e `SVG`, representando elementos visuais numa interface web. 



In [10]:
class Input:
    
    def __init__(self, label: str = ""):
        self.value = ""
        self.label = label
        
    def change(self, char: str):
        self.value += char
    

class SVG:
    
    def __init__(self, path : str, title: str = "", description: str = ""):
        self.path = path
        self.title = title
        self.description = description 

class Image:
    
    def __init__(self, src: str, width: int, height: int, alt_text: str = ""):
        self.src = src
        self.width = width
        self.height = height
        self.alt_text = alt_text
        

username = Input()
password = Input()
diagram = Image("http://coursegalaxy.com/designpatterns/visitor/images/visitor_structure.jpg", 300, 400)
circle = SVG("d=\"M cx cy m -r, 0 a r,r 0 1,0 (r * 2),0 a r,r 0 1,0 -(r * 2),0\"")
username.change("a")
username.change("d")
username.change("m")
username.change("i")
username.change("n")
password.change("a")
password.change("d")
password.change("m")
password.change("i")
password.change("n")

print(username.value)
print(password.value)

admin
admin


As propriedades descritivas como `label`, `alt_text` e `title` sao opcionais em nossas classes. Suponhamos que decidimos nos preocupar com a acessibilidade da nossa interface e queremos saber quais objetos sao acessíveis e quais nao sao. Uma maneira de fazer isso seria acrescentar um método `is_accessible` a cada classe para verificar se o objeto cumpre com regras de acessibilidade. Porém, isso seria dar ao objeto uma funcao que nao tem a ver com sua responsabilidade principal. Se amanha decidirmos adicionar a funcionalidade `render_as_HTML_string`, que imprime os elementos HTML como string, teríamos que adicionar novos métodos a todos os objetos -- e novas responsabilidades. O padrao Visitor permite que essas funcionalidades adicionais sejam responsabilidade de uma classe externa, sendo que a classe original só precisa aceitar uma "visita".

In [13]:
import abc

class HTMLVisitor(abc.ABC):
    
    @abc.abstractmethod
    def visit(self, html_element: "HTMLElement"):
        pass


class HTMLElement(abc.ABC):
    
    @abc.abstractmethod
    def accept(self, visitor: HTMLVisitor):
        pass

    
class Input(HTMLElement):
    
    def __init__(self, label: str = ""):
        self.value = ""
        self.label = label
        
    def change(self, char: str):
        self.value += char
    
    def accept(self, visitor: HTMLVisitor):
        visitor.visitInput(self)
    

class SVG(HTMLElement):
    
    def __init__(self, path : str, title: str = "", description: str = ""):
        self.path = path
        self.title = title
        self.description = description 
        
    def accept(self, visitor: HTMLVisitor):
        visitor.visitSVG(self)

        
class Image(HTMLElement):
    
    def __init__(self, src: str, width: int, height: int, alt_text: str = ""):
        self.src = src
        self.width = width
        self.height = height
        self.alt_text = alt_text
        
    def accept(self, visitor: HTMLVisitor):
        visitor.visitImage(self)
        

class AccessibilityVisitor(HTMLVisitor):
    """
    Checks if an HTML object is accessible
    """

    def visit(self, html_element: HTMLElement):
        html_element.accept(self)
        
    def visitSVG(self, svg: SVG):
        print(bool(svg.title) and bool(svg.description))
    
    def visitImage(self, image: Image):
        print(bool(image.alt_text))
    
    def visitInput(self, textbox: Input):
        print(bool(textbox.label))
 
# Testing the accessibility visitor 

accessibility_visitor = AccessibilityVisitor()

textbox = Input()
accessible_textbox = Input("Some label")
diagram = Image("http://coursegalaxy.com/designpatterns/visitor/images/visitor_structure.jpg", 300, 400)
accessible_diagram = Image("http://coursegalaxy.com/designpatterns/visitor/images/visitor_structure.jpg", 300, 400, "A diagram")
circle = SVG("d=\"M cx cy m -r, 0 a r,r 0 1,0 (r * 2),0 a r,r 0 1,0 -(r * 2),0\"")
accessible_circle = SVG("d=\"M cx cy m -r, 0 a r,r 0 1,0 (r * 2),0 a r,r 0 1,0 -(r * 2),0\"", "A circle", "It's nice")

accessibility_visitor.visit(textbox)
accessibility_visitor.visit(accessible_textbox)
accessibility_visitor.visit(diagram)
accessibility_visitor.visit(accessible_diagram)
accessibility_visitor.visit(circle)
accessibility_visitor.visit(accessible_circle)


False
True
False
True
False
True


Como a responsabilidade de invocar o método correto do Visitor cabe ao método `accept` de cada classe, é possível usar o mesmo Visitor para iterar sobre todos os elementos de uma estrutura de dados, realizando a operacao correta em cada um deles. 

In [5]:
page = [textbox, accessible_textbox, diagram, accessible_diagram, circle, accessible_circle]

for element in page:
    accessibility_visitor.visit(element)


False
True
False
True
False
True


Também podemos adicionar novos visitors sem fazer qualquer mudanca à classe original:

In [17]:
class HTMLStringify(HTMLVisitor):
    """
    Prints a string version of an HTMLElement
    """

    def visit(self, html_element: HTMLElement):
        html_element.accept(self)
        
    def visitSVG(self, svg: SVG):
        html_string = "<svg>"
        html_string += bool(svg.title)*f"\n\t<title>{svg.title}</title>"
        html_string += bool(svg.description)*f"\n\t<description>{svg.description}</description>"
        html_string += f"\n\t<path {svg.path} />"
        html_string += "\n</svg>"
        print(html_string)
        
    def visitImage(self, image: Image):
        html_string = f"<img src=\"{image.src}\" width=\"{image.width}\" height=\"{image.height}\""
        html_string += bool(image.alt_text)*f" alt=\"{image.alt_text}\""
        html_string += " />"
        print(html_string)
    
    def visitInput(self, textbox: Input):
        html_string = ""
        html_string += bool(textbox.label) * f"<label>{textbox.label}\n\t"
        html_string += "<input type=\"text\" />"
        html_string += bool(textbox.label) * "\n</label>"
        print(html_string)
    

stringifier = HTMLStringify()
page = [textbox, accessible_textbox, diagram, accessible_diagram, circle, accessible_circle]

for element in page:
    stringifier.visit(element)


<input type ="text" />
<label>Some label
	<input type ="text" />
</label>
<img src="http://coursegalaxy.com/designpatterns/visitor/images/visitor_structure.jpg" width="300" height="400" />
<img src="http://coursegalaxy.com/designpatterns/visitor/images/visitor_structure.jpg" width="300" height="400" alt="A diagram" />
<svg>
	<path d="M cx cy m -r, 0 a r,r 0 1,0 (r * 2),0 a r,r 0 1,0 -(r * 2),0" />
</svg>
<svg>
	<title>A circle</title>
	<description>It's nice</description>
	<path d="M cx cy m -r, 0 a r,r 0 1,0 (r * 2),0 a r,r 0 1,0 -(r * 2),0" />
</svg>


## Prós e contras:

### Prós:

- Permite que a classe que recebe a visita mantenha apenas uma responsabilidade
- Evita grandes mudancas na classe que recebe a visita: é preciso somente adicionar o método `accept` quando o primeiro Visitor for implementado
- Útil para realizar operacoes sobre diversos objetos numa estrutura. 

### Contras: 

- É necessário criar e manter um método no Visitor para cada uma das classes visitadas
- Caso o objeto tenha propriedades privadas, o Visitor nao teria acesso a elas

## Discussao:
One issue with the Visitor pattern involves cyclicality. When you add a new Visitor, you must make changes to existing code. How would you work around this possible problem?